In [9]:
import os

# GDV


In [8]:
# Assuming you have already defined the computeGDV function in your code
# ROUTINES FOR COMPUTING THE GENERAL DISCRIMINATION VALUE

'''
The routine cmpGDV(dta,lab) expects
  a matrix of data (rows = data vectors) and
  a vector of corresponding labels.

It returns
  the mean intra-cluster distance,
  the mean inter-cluster distance, and
  the gdv-value
'''

# ***** IMPORTS ****************************************************************

from numpy import unique, concatenate, zeros
from numpy import isnan, isinf, sum, sqrt, array, triu
from numpy.random import seed, multivariate_normal
from scipy.spatial import distance

# ------------------------------------------------------------------------------

def makeGDVData(dta,lab):
  res = []
  labels = unique(lab)
  for L in labels:
    res.append( dta[ lab==L ] )
  return res

# ------------------------------------------------------------------------------

def zScoreSpecial(data):

  # get parameters
  NC = len(data) # nr. of clusters
  ND = data[0].shape[1] # nr. of dimensions

  # copy data --> zData
  zData = []
  for C in range(NC):
    arr = data[C].copy()
    zData.append(arr)

  # compute means and STDs for each dimension, over ALL data
  all = concatenate(zData)
  mu =  zeros(shape=ND, dtype=float)
  sig = zeros(shape=ND, dtype=float)
  for D in range(ND):
    mu[D]  = all[:,D].mean()
    sig[D] = all[:,D].std()

  # z-score the data in each cluster
  for C in range(NC):
    for D in range(ND):
      zData[C][:,D] = ( zData[C][:,D] - mu[D] ) / ( 2 * sig[D] )

  # replace nan and inf by 0
  for C in range(NC):
    nanORinf = isnan(zData[C]) | isinf(zData[C])
    zData[C][ nanORinf ] = 0.0

  return zData

# ------------------------------------------------------------------------------

def computeGDV(data):

  '''
  Returns the Generalized Discrimination Value
  as well as intraMean and interMean

  data is expected to be a list of label-sorted point 'clusters':
  data = [cluster1, cluster2, ...]

  Each cluster is a NumPy matrix,
  and the rows of this matrix
  are n-dimensional data vectors,
  each belonging to the same label.
  '''

  # get parameters
  NC = len(data) # nr. of clusters
  ND = data[0].shape[1] # nr. of dimensions

  # copy data --> zData
  zData = []
  for C in range(NC):
    arr = data[C].copy()
    zData.append(arr)

  # dimension-wise z-scoring
  zData = zScoreSpecial(zData)

  # intra-cluster distances
  dIntra = zeros(shape=NC, dtype=float)
  for C in range(NC):
    NP = zData[C].shape[0]
    dis = distance.cdist(zData[C], zData[C], 'euclidean')
    # dis is symmetric with zero diagonal
    dIntra[C] = sum(dis) / (NP*(NP-1)) # divide by nr. of non-zero el.
  #print('dIntra = ',dIntra)

  # inter-cluster distances
  dInter = zeros(shape=(NC,NC), dtype=float)
  for C1 in range(NC):
    NP1 = zData[C1].shape[0]
    for C2 in range(NC):
      NP2 = zData[C2].shape[0]
      dis = distance.cdist(zData[C1], zData[C2], 'euclidean')
      dInter[C1][C2] = sum(dis) / (NP1*NP2) # divide by nr. of non-zero el.
  #print('dInter =\n',dInter)

  # compute GDV
  pre = 1.0 / sqrt(float(ND))
  intraMean = dIntra.mean()
  interMean = sum( triu(dInter,k=1) ) / (NC*(NC-1)/2) # divide by nr. of non-zero el.
  #print('intraMean=',intraMean,'\ninterMean=',interMean)
  gdv = pre * (intraMean - interMean)

  return pre*intraMean, pre*interMean,gdv

# ------------------------------------------------------------------------------

def cmpGDV(dta,lab):
  gdvData = makeGDVData(dta,lab)
  intraMean,interMean,gdv = computeGDV(gdvData)
  return intraMean,interMean,gdv

# ------------------------------------------------------------------------------

def TestGDV():

  # TEST 1

  # generate first cluster
  mean = array([0.0, 0.0])
  cov = array([[0.04, 0.0 ],
               [0.0 , 0.04]])
  seed(978820)
  cluster1 =  multivariate_normal(mean,cov,1000)
  print(cluster1)

  # generate second cluster
  mean = array([1.0, 1.0])
  cov = array([[0.04, 0.0 ],
               [0.0 , 0.04]])
  seed(978820)
  cluster2 =  multivariate_normal(mean,cov,1000)

  # data = list of clusters
  data = []
  data.append(cluster1)
  data.append(cluster2)
  #Plot2D(data,0,1,'case1.png')

  # compute GDV
  intraMean,interMean,gdv = computeGDV(data)
  print('GDV = ',gdv)

  # TEST 2

  # generate first cluster
  mean = array([0.0, 0.0])
  cov = array([[1.0, 0.0 ],
               [0.0 ,1.0]])
  seed(978820)
  cluster1 =  multivariate_normal(mean,cov,1000)

  # generate second cluster
  mean = array([1.0, 1.0])
  cov = array([[1.0, 0.0 ],
               [0.0 , 1.0]])
  seed(978820)
  cluster2 =  multivariate_normal(mean,cov,1000)

  # data = list of clusters
  data = []
  data.append(cluster1)
  data.append(cluster2)
  #Plot2D(data,0,1,'case1.png')

  # compute GDV
  intraMean,interMean,gdv = computeGDV(data)
  print('GDV = ',gdv)

# Data

In [10]:
import numpy as np
import os

# List of .npz file paths (layers 1-12)/ for the CxG-bert only change the paths in "file_paths".
file_paths = [
    'CxG_layer_01_2-10000.npz',
    'CxG_layer_02_2-10000.npz',
    'CxG_layer_03_2-10000.npz',
    'CxG_layer_04_2-10000.npz',
    'CxG_layer_05_2-10000.npz',
    'CxG_layer_06_2-10000.npz',
    'CxG_layer_07_2-10000.npz',
    'CxG_layer_08_2-10000.npz',
    'CxG_layer_09_2-10000.npz',
    'CxG_layer_10_2-10000.npz',
    'CxG_layer_11_2-10000.npz',
    'CxG_layer_12_2-10000.npz'
]

# Loop through each file and check what it contains
for file in file_paths:
    print(f"\n--- {file} ---")
    data = np.load(file)
    print("Keys:", data.files)   # list of array names stored in the file
    for key in data.files:
        arr = data[key]
        print(f"  {key}: shape={arr.shape}, dtype={arr.dtype}")


--- CxG_layer_01_2-10000.npz ---
Keys: ['agree_on', 'agree_with', 'agree_that', 'agree_to', 'come_back', 'come_in', 'come_out', 'give_up', 'give_in', 'give_out', 'give_away']
  agree_on: shape=(100, 768), dtype=float32
  agree_with: shape=(100, 768), dtype=float32
  agree_that: shape=(100, 768), dtype=float32
  agree_to: shape=(100, 768), dtype=float32
  come_back: shape=(100, 768), dtype=float32
  come_in: shape=(100, 768), dtype=float32
  come_out: shape=(100, 768), dtype=float32
  give_up: shape=(100, 768), dtype=float32
  give_in: shape=(99, 768), dtype=float32
  give_out: shape=(93, 768), dtype=float32
  give_away: shape=(100, 768), dtype=float32

--- CxG_layer_02_2-10000.npz ---
Keys: ['agree_on', 'agree_with', 'agree_that', 'agree_to', 'come_back', 'come_in', 'come_out', 'give_up', 'give_in', 'give_out', 'give_away']
  agree_on: shape=(100, 768), dtype=float32
  agree_with: shape=(100, 768), dtype=float32
  agree_that: shape=(100, 768), dtype=float32
  agree_to: shape=(100, 768

# All and within group family

In [5]:
# verb groups
groups = {
    "agree": ['agree_on', 'agree_with', 'agree_that', 'agree_to'],
    "come":  ['come_back', 'come_in', 'come_out'],
    "give":  ['give_up', 'give_in', 'give_out', 'give_away'],
    "all":   ['agree_on', 'agree_with', 'agree_that', 'agree_to',
              'come_back', 'come_in', 'come_out',
              'give_up', 'give_in', 'give_out', 'give_away'],
}

# ---- assumes cmpGDV(embeddings, labels) is already defined in scope ----
# from GDV code above

for file_path in file_paths:
    data = np.load(file_path)
    file_title = os.path.basename(file_path).replace('.npz', '')
    print(f'\n=== {file_title} ===')

    for group_name, keys in groups.items():
        embeddings = []
        labels = []
        label_id = 0

        # Collect embeddings for this group only (and keep class IDs 0..K-1)
        for key in keys:
            if key in data.files:
                arr = data[key]
                embeddings.append(arr)
                labels.extend([label_id] * len(arr))
                label_id += 1
            else:
                # key missing in this layer file (just skip it)
                pass

        # Need at least 2 classes to compute inter-cluster distances
        if label_id < 2:
            print(f'  [{group_name}] Not enough classes present to compute GDV (found {label_id}).')
            continue

        # Stack and compute GDV
        embeddings = np.vstack(embeddings)
        labels = np.array(labels, dtype=int)

        intra_mean, inter_mean, gdv = cmpGDV(embeddings, labels)

        # Print results
        print(f'  [{group_name}]')
        print(f'    Intra-cluster mean distance = {intra_mean:.3f}')
        print(f'    Inter-cluster mean distance = {inter_mean:.3f}')
        print(f'    GDV = {gdv:.3f}')


=== CxG_layer_01_2-10000 ===
  [agree]
    Intra-cluster mean distance = 0.690
    Inter-cluster mean distance = 0.707
    GDV = -0.017
  [come]
    Intra-cluster mean distance = 0.676
    Inter-cluster mean distance = 0.715
    GDV = -0.039
  [give]
    Intra-cluster mean distance = 0.671
    Inter-cluster mean distance = 0.713
    GDV = -0.041
  [all]
    Intra-cluster mean distance = 0.377
    Inter-cluster mean distance = 0.706
    GDV = -0.329

=== CxG_layer_02_2-10000 ===
  [agree]
    Intra-cluster mean distance = 0.686
    Inter-cluster mean distance = 0.710
    GDV = -0.024
  [come]
    Intra-cluster mean distance = 0.688
    Inter-cluster mean distance = 0.712
    GDV = -0.023
  [give]
    Intra-cluster mean distance = 0.681
    Inter-cluster mean distance = 0.711
    GDV = -0.030
  [all]
    Intra-cluster mean distance = 0.432
    Inter-cluster mean distance = 0.710
    GDV = -0.278

=== CxG_layer_03_2-10000 ===
  [agree]
    Intra-cluster mean distance = 0.687
    Inter-cl

per layer × per group with permutation test + plots

In [7]:
import os
import numpy as np
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from numpy.random import default_rng

# ------------------------------------------------------------
# The existing GDV definition uses a special z-score and Euclidean distances.
# Here we reproduce that logic but vectorized and with a single distance matrix.
# ------------------------------------------------------------

def zscore_special_matrix(X):
    """
    Reproduce zScoreSpecial() used by GDV:
      - subtract global mean per dim
      - divide by 2*std per dim
      - replace nan/inf with 0
    """
    mu = X.mean(axis=0)
    sig = X.std(axis=0)
    Z = (X - mu) / (2.0 * (sig + 1e-12))
    Z[~np.isfinite(Z)] = 0.0
    return Z

def gdv_from_distance_matrix(D, y, ndims):
    """
    Compute (pre*intraMean, pre*interMean, gdv) from:
      - D: full pairwise distances (square matrix)
      - y: integer class labels (shape [N])
      - ndims: embedding dimensionality (for the pre-factor 1/sqrt(ndims))
    Mirrors your computeGDV() definition.
    """
    pre = 1.0 / np.sqrt(float(ndims))
    classes = np.unique(y)
    k = len(classes)

    # Intra-cluster means
    intra_vals = []
    for c in classes:
        idx = np.where(y == c)[0]
        if len(idx) < 2:
            # cannot compute intra distances for singleton class; skip
            continue
        d_sub = D[np.ix_(idx, idx)]
        # exclude diagonal -> average of upper triangle
        n = len(idx)
        tri = d_sub[np.triu_indices(n, k=1)]
        intra_vals.append(tri.mean() if tri.size > 0 else 0.0)
    intra_mean = np.mean(intra_vals) if len(intra_vals) > 0 else 0.0

    # Inter-cluster mean (average over all unordered class pairs)
    inter_vals = []
    for i in range(k):
        for j in range(i+1, k):
            idx_i = np.where(y == classes[i])[0]
            idx_j = np.where(y == classes[j])[0]
            if len(idx_i) == 0 or len(idx_j) == 0:
                continue
            block = D[np.ix_(idx_i, idx_j)]
            inter_vals.append(block.mean())
    inter_mean = np.mean(inter_vals) if len(inter_vals) > 0 else 0.0

    gdv = pre * (intra_mean - inter_mean)
    return pre*intra_mean, pre*inter_mean, gdv

def observed_gdv(X, y):
    """
    Compute observed GDV using the same scaling as your cmpGDV:
    - z-score (special)
    - Euclidean distances
    """
    ndims = X.shape[1]
    Z = zscore_special_matrix(X)
    D = squareform(pdist(Z, metric='euclidean'))
    return gdv_from_distance_matrix(D, y, ndims), D

def permutation_test_gdv(D, y, ndims, n_perm=10_000, seed=0):
    """
    Permute labels and recompute GDV n_perm times.
    Returns:
      gdv_obs, gdv_null (array shape [n_perm]),
      rank (1=smallest), p_rank = rank/(n_perm+1)
    NOTE: More negative GDV indicates *stronger* separability,
          so we compute rank in ASCENDING order (most negative first),
          as in Krauss et al. (Pr(X < Δ0)). :contentReference[oaicite:1]{index=1}
    """
    rng = default_rng(seed)
    y = np.asarray(y)
    (_, _, gdv_obs) = gdv_from_distance_matrix(D, y, ndims)

    gdv_null = np.empty(n_perm, dtype=float)
    for r in range(n_perm):
        y_perm = rng.permutation(y)
        (_, _, gdv_p) = gdv_from_distance_matrix(D, y_perm, ndims)
        gdv_null[r] = gdv_p

    # Combine {null + observed}, rank ascending (1..n_perm+1)
    all_vals = np.concatenate([gdv_null, np.array([gdv_obs])])
    order = np.argsort(all_vals)
    # position of the observed value in the sorted array:
    rank = int(np.where(order == n_perm)[0][0]) + 1  # 1-based index
    p_rank = rank / (n_perm + 1.0)

    return gdv_obs, gdv_null, rank, p_rank

def plot_null_with_observed(gdv_null, gdv_obs, title=None, bins=60):
    """
    Plot histogram of null GDVs and mark the observed GDV.
    """
    plt.figure(figsize=(7,4.2))
    plt.hist(gdv_null, bins=bins, alpha=0.8, edgecolor='k')
    plt.axvline(gdv_obs, linestyle='--', linewidth=2)
    if title:
        plt.title(title)
    plt.xlabel('GDV (null via label permutations)')
    plt.ylabel('Count')
    plt.tight_layout()

# ------------------------------------------------------------
# Data loading helpers and groups
# ------------------------------------------------------------

def load_family_from_npz(npz_path, keys_in_family):
    """
    Load embeddings and integer labels (0..C-1) for a given family in a given layer (.npz).
    Skips missing keys gracefully. Returns (X, y, class_names).
    """
    d = np.load(npz_path)
    X_list, y_list, names = [], [], []
    label_id = 0
    for key in keys_in_family:
        if key in d.files:
            Xi = d[key]
            X_list.append(Xi)
            y_list.append(np.full(len(Xi), label_id, dtype=int))
            names.append(key)
            label_id += 1
    if label_id < 2:
        return None, None, names  # not enough classes to compute GDV
    X = np.vstack(X_list)
    y = np.concatenate(y_list)
    return X, y, names

file_paths = [f'CxG_layer_{i:02d}_2-10000.npz' for i in range(1, 13)]
groups = {
    "agree": ['agree_on', 'agree_with', 'agree_that', 'agree_to'],
    "come":  ['come_back', 'come_in', 'come_out'],
    "give":  ['give_up', 'give_in', 'give_out', 'give_away'],
    "all":   ['agree_on', 'agree_with', 'agree_that', 'agree_to',
              'come_back', 'come_in', 'come_out',
              'give_up', 'give_in', 'give_out', 'give_away'],
}

# ------------------------------------------------------------
# MAIN LOOP: per layer × per group
# ------------------------------------------------------------

# You can reduce n_perm while testing (e.g., 100 or 1000), then set to 10000 for the paper.
N_PERM = 10_000
RNG_SEED = 42

results = []  # collect (layer, group, gdv_obs, p_rank, rank, gdv_null_summary)

for npz_path in file_paths:
    layer_name = os.path.basename(npz_path).replace('.npz', '')
    print(f'\n=== {layer_name} ===')

    for group_name, family_keys in groups.items():
        X, y, class_names = load_family_from_npz(npz_path, family_keys)
        if X is None or len(np.unique(y)) < 2:
            print(f'  [{group_name}] Not enough classes to compute GDV (found {len(class_names)} present).')
            continue

        # Observed GDV and distance matrix
        (intra_obs, inter_obs, gdv_obs), D = observed_gdv(X, y)

        # Permutation test (rank-based p exactly as requested)
        ndims = X.shape[1]
        gdv_obs_val, gdv_null, rank, p_rank = permutation_test_gdv(D, y, ndims, n_perm=N_PERM, seed=RNG_SEED)

        # Print summary
        print(f'  [{group_name}]')
        print(f'    Classes: {class_names}')
        print(f'    Intra = {intra_obs:.3f} | Inter = {inter_obs:.3f} | GDV_obs = {gdv_obs:.3f}')
        print(f'    Permutations = {N_PERM} | Rank = {rank} of {N_PERM+1} | p = {p_rank:.5f}')

        # Store results (optional)
        results.append((layer_name, group_name, gdv_obs, p_rank, rank,
                        float(np.mean(gdv_null)), float(np.std(gdv_null))))

        # Plot null distribution with observed marker and save per (layer, group)
        title = f'{layer_name} – {group_name} | GDV null vs observed'
        plot_null_with_observed(gdv_null, gdv_obs, title=title, bins=60)
        out_png = f'gdv_null_{layer_name}_{group_name}.png'
        plt.savefig(out_png, dpi=200)
        plt.close()

# Optional: print a compact table at the end
print("\nSummary (layer, group, GDV_obs, p_rank, rank, null_mean, null_sd):")
for row in results:
    print(row)


=== CxG_layer_01_2-10000 ===
  [agree]
    Classes: ['agree_on', 'agree_with', 'agree_that', 'agree_to']
    Intra = 0.690 | Inter = 0.707 | GDV_obs = -0.017
    Permutations = 10000 | Rank = 1 of 10001 | p = 0.00010
  [come]
    Classes: ['come_back', 'come_in', 'come_out']
    Intra = 0.676 | Inter = 0.715 | GDV_obs = -0.039
    Permutations = 10000 | Rank = 1 of 10001 | p = 0.00010
  [give]
    Classes: ['give_up', 'give_in', 'give_out', 'give_away']
    Intra = 0.671 | Inter = 0.713 | GDV_obs = -0.041
    Permutations = 10000 | Rank = 1 of 10001 | p = 0.00010
  [all]
    Classes: ['agree_on', 'agree_with', 'agree_that', 'agree_to', 'come_back', 'come_in', 'come_out', 'give_up', 'give_in', 'give_out', 'give_away']
    Intra = 0.377 | Inter = 0.706 | GDV_obs = -0.329
    Permutations = 10000 | Rank = 1 of 10001 | p = 0.00010

=== CxG_layer_02_2-10000 ===
  [agree]
    Classes: ['agree_on', 'agree_with', 'agree_that', 'agree_to']
    Intra = 0.686 | Inter = 0.710 | GDV_obs = -0.024
 

Save output

In [ ]:
import os, csv, zipfile
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# ---- CONFIG ----
file_paths = [f'CxG_layer_{i:02d}_2-10000.npz' for i in range(1, 13)]
N_PERM = 10_000
RNG_SEED = 42

OUT_CSV = "CxG-BERT-gdv_results_summary.csv"
PLOTS_DIR = "CxG-BERT-base-gdv_null_plots"
OUT_ZIP = "CxG-BERT-gdv_null_plots.zip"

# Make plots dir
os.makedirs(PLOTS_DIR, exist_ok=True)

# Collect rows for CSV
rows = []

for npz_path in file_paths:
    layer_name = os.path.basename(npz_path).replace(".npz", "")

    for group_name, family_keys in groups.items():
        X, y, present = load_family_from_npz(npz_path, family_keys)
        if X is None or len(np.unique(y)) < 2:
            rows.append({
                "layer": layer_name,
                "group": group_name,
                "n_classes": len(present),
                "n_samples": 0,
                "intra_obs": np.nan,
                "inter_obs": np.nan,
                "gdv_obs": np.nan,
                "rank": np.nan,
                "p_rank": np.nan,
                "n_perm": N_PERM,
                "null_mean": np.nan,
                "null_sd": np.nan,
                "null_p2p5": np.nan,
                "null_p50": np.nan,
                "null_p97p5": np.nan,
                "classes_present": ";".join(present),
            })
            continue

        (intra_obs, inter_obs, gdv_obs), D = observed_gdv(X, y)
        ndims = X.shape[1]
        gdv_obs_val, gdv_null, rank, p_rank = permutation_test_gdv(
            D, y, ndims, n_perm=N_PERM, seed=RNG_SEED
        )

        # Plot & save
        title = f'{layer_name} – {group_name} | GDV null vs observed'
        plot_null_with_observed(gdv_null, gdv_obs, title=title, bins=60)
        plot_path = os.path.join(PLOTS_DIR, f'CxG-BERT_gdv_null_{layer_name}_{group_name}.png')
        plt.savefig(plot_path, dpi=200)
        plt.close()

        # Summaries for CSV
        null_mean = float(np.mean(gdv_null))
        null_sd = float(np.std(gdv_null))
        p2p5, p50, p97p5 = np.percentile(gdv_null, [2.5, 50, 97.5])

        rows.append({
            "layer": layer_name,
            "group": group_name,
            "n_classes": len(np.unique(y)),
            "n_samples": X.shape[0],
            "intra_obs": float(intra_obs),
            "inter_obs": float(inter_obs),
            "gdv_obs": float(gdv_obs),
            "rank": int(rank),
            "p_rank": float(p_rank),
            "n_perm": N_PERM,
            "null_mean": null_mean,
            "null_sd": null_sd,
            "null_p2p5": float(p2p5),
            "null_p50": float(p50),
            "null_p97p5": float(p97p5),
            "classes_present": ";".join(present),
        })

# Write CSV
df = pd.DataFrame(rows)
df.sort_values(by=["group", "layer"], inplace=True)
df.to_csv(OUT_CSV, index=False)
print(f"Saved summary to {OUT_CSV} (rows={len(df)})")

# Zip all plots for easy download/sharing
with zipfile.ZipFile(OUT_ZIP, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
    for fname in os.listdir(PLOTS_DIR):
        if fname.lower().endswith(".png"):
            zf.write(os.path.join(PLOTS_DIR, fname),
                     arcname=fname)
print(f"Zipped plots to {OUT_ZIP}")

# Optional: show a small preview of the table
print(df.head(10).to_string(index=False))

In [9]:
# === Setup ===
import os, re, zipfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr

# If your file is in Colab, set the path accordingly (e.g., /content/gdv_results_summary.csv)
CSV_PATH = "/content/CxG-BERT-gdv_results_summary.csv"  # change if needed

# === Load ===
df = pd.read_csv(CSV_PATH)

# Extract numeric layer index
def layer_index(name: str) -> int:
    m = re.search(r"layer_(\d+)", name)
    return int(m.group(1)) if m else 0

df["layer_idx"] = df["layer"].apply(layer_index)
df_valid = df.dropna(subset=["gdv_obs", "p_rank"]).copy()

# === Benjamini–Hochberg FDR ===
def bh_fdr(pvals):
    p = np.asarray(pvals, dtype=float)
    n = p.size
    order = np.argsort(p)
    ranked = p[order]
    q = ranked * n / (np.arange(n) + 1)
    q = np.minimum.accumulate(q[::-1])[::-1]  # monotone
    qvals = np.empty_like(q)
    qvals[order] = q
    return np.clip(qvals, 0, 1)

# FDR across all tests
df_valid["q_fdr_all"] = bh_fdr(df_valid["p_rank"].values)

# FDR within each group (optional, informative)
df_valid["q_fdr_group"] = np.nan
for g, gdf in df_valid.groupby("group"):
    qg = bh_fdr(gdf["p_rank"].values)
    df_valid.loc[gdf.index, "q_fdr_group"] = qg

# Merge back and save
df_out = df.merge(df_valid[["layer","group","q_fdr_all","q_fdr_group"]],
                  on=["layer","group"], how="left")
df_out.to_csv("CxG-BERT-gdv_results_with_fdr.csv", index=False)

# === Plots ===
os.makedirs("plots", exist_ok=True)
png_paths = []

# 1) Observed GDV by layer (per group)
for g, gdf in df_valid.groupby("group"):
    gdf = gdf.sort_values("layer_idx")
    plt.figure(figsize=(7,4.2))
    plt.plot(gdf["layer_idx"], gdf["gdv_obs"], marker="o")
    plt.xlabel("Layer")
    plt.ylabel("Observed GDV")
    plt.title(f"Observed GDV by layer — {g}")
    plt.tight_layout()
    out = f"plots/gdv_gdvobs_by_layer_{g}.png"
    plt.savefig(out, dpi=200)
    plt.close()
    png_paths.append(out)

# 2) Permutation p-values by layer (per group)
for g, gdf in df_valid.groupby("group"):
    gdf = gdf.sort_values("layer_idx")
    plt.figure(figsize=(7,4.2))
    plt.plot(gdf["layer_idx"], gdf["p_rank"], marker="o")
    plt.axhline(0.05, linestyle="--")  # visual threshold
    plt.xlabel("Layer")
    plt.ylabel("Permutation p-value")
    plt.title(f"Permutation p by layer — {g}")
    plt.tight_layout()
    out = f"plots/gdv_pval_by_layer_{g}.png"
    plt.savefig(out, dpi=200)
    plt.close()
    png_paths.append(out)

# Zip the plots (easy to download)
with zipfile.ZipFile("CxG-BERT_gdv_plots.zip", "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for p in png_paths:
        zf.write(p, arcname=os.path.basename(p))

# === Auto Results text ===
lines = []
lines.append("Permutation-based assessment of GDV separability\n")
lines.append("We computed GDV per layer and verb group and assessed significance via 10,000 label permutations.\n")

for g, gdf in df_valid.groupby("group"):
    gdf = gdf.sort_values("layer_idx")
    rho, rho_p = spearmanr(gdf["layer_idx"].values, gdf["gdv_obs"].values)
    i_min = gdf["gdv_obs"].idxmin()
    row_min = df_valid.loc[i_min]
    best_layer = int(row_min["layer_idx"])
    best_gdv = float(row_min["gdv_obs"])
    best_p = float(row_min["p_rank"])
    sig_any = int(np.sum((df_valid.loc[df_valid["group"]==g, "q_fdr_all"] <= 0.05).values))
    n_total = int(np.sum(df_valid["group"]==g))
    lines.append(f"Group '{g}': Spearman ρ={rho:.3f} (p={rho_p:.3g}); "
                 f"strongest separation at layer {best_layer} (GDV={best_gdv:.3f}, p={best_p:.3g}). "
                 f"FDR (all-tests) significant layers: {sig_any}/{n_total}.\n")

if "all" in df_valid["group"].unique():
    gdf = df_valid[df_valid["group"]=="all"].sort_values("layer_idx")
    rho, rho_p = spearmanr(gdf["layer_idx"].values, gdf["gdv_obs"].values)
    i_min = gdf["gdv_obs"].idxmin(); row_min = df_valid.loc[i_min]
    best_layer = int(row_min["layer_idx"])
    best_gdv = float(row_min["gdv_obs"]); best_p = float(row_min["p_rank"])
    sig_any = int(np.sum((gdf["q_fdr_all"] <= 0.05).values)); n_total = gdf.shape[0]
    lines.append(f"All constructions pooled: Spearman ρ={rho:.3f} (p={rho_p:.3g}); "
                 f"strongest separation at layer {best_layer} (GDV={best_gdv:.3f}, p={best_p:.3g}). "
                 f"FDR significant layers: {sig_any}/{n_total}.\n")

with open("gdv_results_summary.txt", "w") as f:
    f.writelines(lines)

print("Wrote: gdv_results_with_fdr.csv, plots/*.png, gdv_plots.zip, gdv_results_summary.txt")

Wrote: gdv_results_with_fdr.csv, plots/*.png, gdv_plots.zip, gdv_results_summary.txt
